In [150]:
import os

In [151]:
tavily_api_key = os.environ.get("TAVILY_API_KEY")
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

search_en_id = os.getenv('search_en_id')
api_key = os.getenv('api_key')

In [152]:
!export LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
!export LANGCHAIN_TRACING_V2=true
!export LANGCHAIN_API_KEY="ls__b338d581f71b48b099c981bf36b095b6"

!export LANGCHAIN_PROJECT="test"

In [153]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings


from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma


In [154]:
import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema import Document

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

In [155]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
import requests

In [156]:
# fixes a bug with asyncio and jupyter
import nest_asyncio

nest_asyncio.apply()

In [157]:
url_ = "https://www.googleapis.com/customsearch/v1"

def build_params(search_query, num = 10, start=1, dateRestrict='d1',**kwargs):
	params = { 'q': search_query, 
		   	   'key': api_key, 
			   'cx': search_en_id,
			   'num': num,
			   'start': 1,
			   'dateRestrict': 'd1',
			   }
	params.update(kwargs)
	return params

def google_search(params):
	response = requests.get(url_, params=params)
	if response.status_code != 200:
		raise Exception('API response: {}'.format(response.status_code))
	return response.json()



In [158]:
def web_search_(state):
	"""
	initial Web search based on the re-phrased question using Google API.

	Args:
		state (dict): The current graph state

	Returns:
		state (dict): Web results appended to documents.
	"""
	
	print("---initial WEB SEARCH---")
	state_dict = state["keys"]
	question = state_dict["question"]
	# documents = state_dict["documents"]

	
	
	response = google_search(build_params(question,num = 5))
	urls = []
	for i in response['items']:
		url, title = i['link'], i['title']
		print(f"URL : {url}  title : {title}\n")
		urls.append(url)
	loader = WebBaseLoader(urls)
	# print(f'Docs: {docs}\n\n')
	# print(f'Docs Type: {type(docs)}\n\n')
	loader.requests_per_second = 1
	docs = loader.aload()
	# state_dict["documents"] = docs
	# Split
	text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
		chunk_size=500, chunk_overlap=100
	)
	all_splits = text_splitter.split_documents(docs)
	
	# Embed and index
	
	embedding = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
	
	
	# Index
	vectorstore = Chroma.from_documents(
		documents=all_splits,
		collection_name="rag-chroma",
		embedding=embedding,
	)
	retriever = vectorstore.as_retriever()
	
	
    
	return {"keys": {"question": question, "retriever": retriever}}

In [159]:
# # Split
# text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
#     chunk_size=500, chunk_overlap=100
# )
# all_splits = text_splitter.split_documents(docs)

# # Embed and index

# embedding = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")


# # Index
# vectorstore = Chroma.from_documents(
#     documents=all_splits,
#     collection_name="rag-chroma",
#     embedding=embedding,
# )
# retriever = vectorstore.as_retriever()

In [160]:
# response = google_search(build_params(q,num = 5))

# urls = []
# for i in response['items']:
# 	url, title = i['link'], i['title']
# 	print(f"URL : {url}  title : {title}\n")
# 	urls.append(url)

In [161]:
# # loader = WebBaseLoader(["https://www.espn.com/", "https://google.com"])
# url = "https://www.simplilearn.com/tutorials/artificial-intelligence-tutorial/rational-agent-in-ai"
# url = "https://lilianweng.github.io/posts/2023-06-23-agent/"
# loader = WebBaseLoader(url)
# loader.requests_per_second = 1
# docs = loader.aload()
# docs

In [162]:
from typing import Annotated, Dict, TypedDict

from langchain_core.messages import BaseMessage


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        keys: A dictionary where each key is a string.
    """

    keys: Dict[str, any]

In [163]:
safety = safety_settings={
	HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
	HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
	HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
	HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
	}

In [164]:
def retrieve(state):
    """
    Retrieve documents
    
    Args:
        state (dict): The current graph state
    
    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    retriever = state_dict["retriever"]
    
    documents = retriever.get_relevant_documents(question)
    return {"keys": {"documents": documents, "question": question}}

In [165]:
def generate(state):
	"""
	Generate answer

	Args:
		state (dict): The current graph state

	Returns:
		state (dict): New key added to state, generation, that contains generation
	"""
	print("---GENERATE---")
	state_dict = state["keys"]
	question = state_dict["question"]
	documents = state_dict["documents"]
	

	# Prompt
	prompt = hub.pull("rlm/rag-prompt")

	# LLM
	llm = ChatGoogleGenerativeAI(model="gemini-pro", 
										verbose=True, 
										temperature=0,
										safety_settings=safety)
	
	# Chain
	rag_chain = prompt | llm | StrOutputParser()

	# Run
	generation = rag_chain.invoke({"context": documents, "question": question})
	return {
		"keys": {"documents": documents, "question": question, "generation": generation}
    }
	
# Post-processing
def format_docs(docs):
	return "\n\n".join(doc.page_content for doc in docs)

In [166]:
def grade_documents(state):
	"""
	Determines whether the retrieved documents are relevant to the question.

	Args:
		state (dict): The current graph state

	Returns:
		state (dict): Updates documents key with relevant documents
	"""

	print("---CHECK RELEVANCE---")
	state_dict = state["keys"]
	question = state_dict["question"]
	documents = state_dict["documents"]

	
	# LLM
	llm = ChatGoogleGenerativeAI(model="gemini-pro", 
										verbose=False, 
										temperature=0,
										safety_settings=safety)

	prompt = PromptTemplate(
		template="""You are a grader assessing relevance of a retrieved document to a user question. \n 
		Here is the retrieved document: \n\n {context} \n\n
		Here is the user question: {question} \n
		If the document contains keywords related to the user question, grade it as relevant. \n
		It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
		Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
		Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.""",
		input_variables=["question", "context"],
	)

	chain = prompt | llm | JsonOutputParser()

	# Score
	filtered_docs = []
	search = "No"  # Default do not opt for web search to supplement retrieval
	for d in documents:
		score = chain.invoke(
			{
				"question": question,
				"context": d.page_content,
			}
		)
		grade = score["score"]
		if grade == "yes":
			print("---GRADE: DOCUMENT RELEVANT---")
			filtered_docs.append(d)
		else:
			print("---GRADE: DOCUMENT NOT RELEVANT---")
			search = "Yes"  # Perform web search
			continue

	return {
		"keys": {
			"documents": filtered_docs,
			"question": question,
			"run_web_search": search,
		}
	}

In [167]:
def transform_query_(state):
    """
    Transform the query to produce a better question.
    
    Args:
        state (dict): The current graph state
    
    Returns:
        state (dict): Updates question key with a re-phrased question
    """
    
    print("---TRANSFORM QUERY---")
    state_dict = state["keys"]
    question = state_dict["question"]
    # documents = state_dict["documents"]
    

    # Create a prompt template with format instructions and the query
    prompt = PromptTemplate(
        template="""You are generating questions that is well optimized for retrieval. \n 
        Look at the input and try to reason about the underlying sematic intent / meaning. \n 
        Here is the initial question:
        \n ------- \n
        {question} 
        \n ------- \n
        Provide an improved question without any premable, only respond with the updated question: """,
        input_variables=["question"],
    )
    
    # Grader
    # LLM
    llm = ChatGoogleGenerativeAI(model="gemini-pro", 
                                        verbose=True, 
                                        temperature=0,
                                        safety_settings=safety)
    
    # Prompt
    chain = prompt | llm | StrOutputParser()
    better_question = chain.invoke({"question": question})

    print(f"Better Question: {better_question}")

    return {
        "keys": { "question": better_question}
    }

In [168]:
def transform_query(state):
    """
    Transform the query to produce a better question.
    
    Args:
        state (dict): The current graph state
    
    Returns:
        state (dict): Updates question key with a re-phrased question
    """
    
    print("---init TRANSFORM QUERY---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]


    # Create a prompt template with format instructions and the query
    prompt = PromptTemplate(
        template="""You are generating questions that is well optimized for retrieval. \n 
        Look at the input and try to reason about the underlying sematic intent / meaning. \n 
        Here is the initial question:
        \n ------- \n
        {question} 
        \n ------- \n
        Provide an improved question without any premable, only respond with the updated question: """,
        input_variables=["question"],
    )
    
    # Grader
    # LLM
    llm = ChatGoogleGenerativeAI(model="gemini-pro", 
                                        verbose=True, 
                                        temperature=0,
                                        safety_settings=safety)
    
    # Prompt
    chain = prompt | llm | StrOutputParser()
    better_question = chain.invoke({"question": question})

    print(f"Better Question: {better_question}")

    return {
        "keys": {"documents": documents, "question": better_question}
    }


def web_search(state):
    """
    Web search based on the re-phrased question using Tavily API.
    
    Args:
        state (dict): The current graph state
    
    Returns:
        state (dict): Web results appended to documents.
    """

    print("---WEB SEARCH---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]
    
    
    tool = TavilySearchResults()
    docs = tool.invoke({"query": question})
    print(f'Docs: {docs}\n\n')
    print(f'Docs Type: {type(docs)}\n\n')
    web_results = "\n".join([d["content"] for d in docs])
    print(web_results)
    web_results = Document(page_content=web_results)
    documents.append(web_results)
    
    return {"keys": {"documents": documents, "question": question}}

In [169]:
### Edges


def decide_to_generate(state):
    """
    Determines whether to generate an answer or re-generate a question for web search.
    
    Args:
        state (dict): The current state of the agent, including all keys.
    
    Returns:
        str: Next node to call
    """

    print("---DECIDE TO GENERATE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    filtered_documents = state_dict["documents"]
    search = state_dict["run_web_search"]
    
    if search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: TRANSFORM QUERY and RUN WEB SEARCH---")
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"

In [170]:
import pprint

from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("transform_query_", transform_query_)  # transform_query
workflow.add_node("web_search_", web_search_)  # Google search
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae
workflow.add_node("transform_query", transform_query)  # transform_query
workflow.add_node("web_search", web_search)  # web search

# Build graph
# workflow.set_entry_point("retrieve")
workflow.set_entry_point("transform_query_")
workflow.add_edge("transform_query_", "web_search_")
workflow.add_edge("web_search_", "retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "web_search")
workflow.add_edge("web_search", "generate")
workflow.add_edge("generate", END)

# Compile
app = workflow.compile()

In [171]:
question =  "Explain how the different types of agent memory work?"

In [172]:
# Run
inputs = {
    "keys": {
        "question": "anime surface tension",
    }
}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint.pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint.pprint("\n---\n")

# Final generation
pprint.pprint(value["keys"]["generation"])

---TRANSFORM QUERY---
Better Question: What is the surface tension of anime?
"Node 'transform_query_':"
'\n---\n'
---initial WEB SEARCH---
URL : https://the-orange-sanders-side.tumblr.com/  title : Orange Side Has Entered The Chat

URL : https://www.reddit.com/r/ClassroomOfTheElite/  title : Welcome to the Classroom of the Elite!

URL : https://gamerant.com/best-winter-2024-anime/  title : Best Winter 2024 Anime, Ranked

URL : https://www.cbr.com/star-trek-lower-decks-like-rick-and-morty/  title : Is Star Trek Lower Decks Like Rick and Morty?

URL : https://en.wikipedia.org/wiki/Mammal  title : Mammal - Wikipedia




Error fetching https://gamerant.com/best-winter-2024-anime/ with attempt 1/3: Server disconnected. Retrying...
Error fetching https://gamerant.com/best-winter-2024-anime/ with attempt 2/3: Server disconnected. Retrying...
Error fetching https://gamerant.com/best-winter-2024-anime/ and aborting, use continue_on_failure=True to continue loading urls after encountering an error.
Traceback (most recent call last):
  File "/home/jrdfm/.local/lib/python3.10/site-packages/langchain_community/document_loaders/web_base.py", line 154, in _fetch_with_rate_limit
    return await self._fetch(url)
  File "/home/jrdfm/.local/lib/python3.10/site-packages/langchain_community/document_loaders/web_base.py", line 131, in _fetch
    async with session.get(
  File "/home/jrdfm/.local/lib/python3.10/site-packages/aiohttp/client.py", line 1194, in __aenter__
    self._resp = await self._coro
  File "/home/jrdfm/.local/lib/python3.10/site-packages/aiohttp/client.py", line 605, in _request
    await resp.start

ServerDisconnectedError: Server disconnected